In [1]:
from shapely.geometry import Point
import seaborn as sns
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

sns.set_style("dark")

In [ ]:
case_count_df=pd.read_csv('data/covid_count.csv')
case_count_df['date']=pd.to_datetime(case_count_df['Specimen Collection Date'])
cases_by_zip=case_count_df.groupby(['Zip']).sum()
cases_by_zip['ZIP'] = cases_by_zip.index
cases_by_zip = cases_by_zip.reset_index()

#needed to change type of zip tp string to drop the decimal and then to an object to join on later
cases_by_zip['ZIP'] = cases_by_zip.loc[:, ['ZIP']].astype(str)
cases_by_zip['ZIP'] = cases_by_zip['ZIP'].str.split('.').str[0]
cases_by_zip['ZIP'] = cases_by_zip.loc[:, ['ZIP']].astype(object)
cases_zip_ord=cases_by_zip.sort_values(by=['Cases'], ascending=False)
cases_zip_ord.head(5)

In [ ]:
top_5_z=cases_zip_ord.head(5)

In [ ]:
top_5_b=cases_zip_ord.tail(5)
top_5_b

In [ ]:
case_count_top=case_count_df[case_count_df['Zip'].isin(top_5_z.Zip)]
case_count_top.head(5)

In [ ]:
case_count_bot=case_count_df[case_count_df['Zip'].isin(top_5_b.Zip)]
case_count_bot.head(5)

In [ ]:

pop_data = pd.read_excel("data/hisp.xlsx", skiprows = 1, header=[0, 1], sheet_name = 0)
pop_data = pop_data[pop_data.columns.drop(list(pop_data.filter(regex='Error')))]
pop_data.columns = pop_data.columns.droplevel(1)
pop_data = pop_data.set_index('Unnamed: 0_level_0').T.reset_index().rename({"index": "zipcode"}, axis = 1)
pop_data['zipcode'] = pop_data.loc[:, ['zipcode']].astype(object)

pop_data.head(5)

In [ ]:
gend_data = pd.read_excel("data/gender_age.xlsx", skiprows = 1, header=[0, 1], sheet_name = 0)
gend_data = gend_data[gend_data.columns.drop(list(gend_data.filter(regex='Error')))]
gend_data.columns = gend_data.columns.droplevel(1)
gend_data = gend_data.set_index('Unnamed: 0_level_0').T.reset_index().rename({"index": "zipcode"}, axis = 1)

In [ ]:
ge_data = pd.read_excel("data/gender_ed.xlsx", skiprows = 1, header=[0, 1], sheet_name = 0)
ge_data = ge_data[ge_data.columns.drop(list(ge_data.filter(regex='Error')))]
ge_data.columns = ge_data.columns.droplevel(1)
ge_data = ge_data.set_index('Unnamed: 0_level_0').T.reset_index().rename({"index": "zipcode"}, axis = 1)

In [ ]:
income_data = pd.read_excel("data/income.xlsx", skiprows = 1, header=[0, 1], sheet_name = 0)
income_data = income_data[income_data.columns.drop(list(income_data.filter(regex='Error')))]
income_data.columns = income_data.columns.droplevel(1)
income_data = income_data.set_index('Unnamed: 0_level_0').T.reset_index().rename({"index": "zipcode"}, axis = 1)

In [ ]:
x=cases_by_zip.merge(income_data, left_on='ZIP', right_on='zipcode', how='left', indicator=True)
y=x.merge(pop_data, left_on='ZIP', right_on='zipcode', how='left', indicator='True')
y=y.drop(columns=['Not Hispanic or Latino:',
       'White alone', 'Black or African American alone',
       'American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some other race alone', 'Two or more races:',
       'Two races including Some other race',
       'Two races excluding Some other race, and three or more races',
       'Hispanic or Latino:', 'White alone', 'Black or African American alone',
       'American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some other race alone', 'Two or more races:',
       'Two races including Some other race',
       'Two races excluding Some other race, and three or more races', 'True', '_merge'])
y.head(5)

In [ ]:
y['pov_pop']=y.iloc[:,5:10].sum(axis=1)
y['pov_perc']=y['pov_pop']/y['Total:_y']*100
y

run this next cell twice. would love a coding solution to not having to say this

In [ ]:
sns.lineplot(data=case_count_top, x="date", y="Cases", hue='Zip', palette='tab10', linewidth=1.5)
mpl.rcParams['figure.figsize'] = 21,12
#sns.set(rc={'axes.facecolor':'black', 'figure.facecolor':'white'})##trying to figure out more settings, not currently working

In [ ]:
sns.set_palette("tab10")
sns.set(style="ticks", rc={"lines.linewidth": 0.9})
g=sns.lineplot(data=case_count_bot, x="date", y="Cases", hue='Zip', palette='tab10')
mpl.rcParams['figure.figsize'] = 20,8.27

In [ ]:
#GB=case_count_df.groupby([(case_count_df.date.year),(case_count_df.date.month)]).sum()

month= case_count_df.groupby([case_count_df['date'].dt.year, 
                                case_count_df['date'].dt.month,
                                case_count_df['Zip']]).agg({'Cases':sum})
month

These next couple charts are mostly tests/"maybe later with more work" kinda options

In [ ]:
sns.pairplot(month_data)

In [ ]:
sns.displot(
    data=case_count_top,
    x="date", y="Cases", hue="Zip",
    kind="kde", height=6,
    multiple="countour", clip=(0, None),
    palette="ch:rot=-.25,hue=1,light=.75")

In [ ]:
sns.displot(
    data=case_count_top,
    x="date", hue="Zip",
    kind="kde", height=6,
    multiple="fill", clip=(0, None),
    palette="ch:rot=-.25,hue=1,light=.75")

In [ ]:
#rs = np.random.RandomState(11)
#x = rs.gamma(2, size=1000)
#y = -.5 * x + rs.normal(size=1000)

#sns.jointplot(data=, x=, y=y, kind="hex", color="#4CB391")

sns.histplot(
    month_data,
    x="date", y="Cases", hue="Zip")

#sns.jointplot(
    #data=case_count_df,
   # x="date", hue="Zip",
   # kind="hex", height=6,
   # multiple="fill", clip=(0, None),
   # palette="ch:rot=-.25,hue=1,light=.75")

In [ ]:
case_count_df=pd.read_csv('data/covid_count.csv')
case_count_df['date']=pd.to_datetime(case_count_df['Specimen Collection Date'])
cases_by_zip=case_count_df.groupby(['Zip']).sum()
cases_by_zip['ZIP'] = cases_by_zip.index
cases_by_zip = cases_by_zip.reset_index()
cases_by_zip['ZIP'] = cases_by_zip.loc[:, ['ZIP']].astype(float)
cases_zip_ord=cases_by_zip.sort_values(by=['Cases'], ascending=False)
cases_zip_ord.head(5)

In [ ]:
vaccine_count_df=pd.read_csv('data/vaccine_with_protocol.csv')
vaccine_count_df['date']=pd.to_datetime(vaccine_count_df['vacc_date'])
vac_by_zip=vaccine_count_df.groupby(['zipcode']).sum()
vac_by_zip['ZIP'] = vac_by_zip.index
vac_by_zip = vac_by_zip.reset_index()
vac_by_zip['ZIP'] = vac_by_zip.loc[:, ['ZIP']].astype(float)
vac_zip_ord=vac_by_zip.sort_values(by=['count_given'], ascending=False)
vac_zip_ord.head(5)

In [ ]:
vaccine_count_df.head(5)

In [ ]:
zipcodes = gpd.read_file('../geospatial-python-workshop/data/zipcodes.geojson')
zipcodes['zip'] = zipcodes.loc[:, ['zip']].astype(float)

In [ ]:
case_zip_map = pd.merge(left = zipcodes, right = cases_by_zip, 
                    left_on = 'zip', right_on = 'ZIP')

#this "coords" shit is something I found online to get the zip labels to work on the map
case_zip_map['coords'] = case_zip_map['geometry'].apply(lambda x: x.representative_point().coords[:])
case_zip_map['coords'] = [coords[0] for coords in case_zip_map['coords']]

case_zip_map.head()